In [1]:
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch.optim as optim


from utils.utils_dataset import *
from utils.utils_unet import *
from utils.utils_training import *
from utils.utils_debug import *



In [2]:


root = '.'

# Retrieve data and include transformation operations
print('Retrieving Data')

train_path_images = root + '/datasets/training/images'
train_path_target = root + '/datasets/training/1st_manual'
train_path_mask = root + '/datasets/training/mask'
test_path = root + '/datasets/test/images'

# transformations

# alternative using Random Operations function
'''
train_transform = transforms.Compose([RandomOperations([RandomHorizontalFlip(p = 1),
                                                        RandomVerticalFlip(p = 1)],
                                                       weights = [.5, .5]),
                                      RandomRotate(degrees = 180),
                                      ToTensor()])
'''
train_transform = transforms.Compose([RandomHorizontalFlip(p = .5),
                                     RandomVerticalFlip(p = .5),
                                     RandomRotate(degrees = 180),
                                     ToTensor()])

test_transform = ToTensor()

# retrieve dataset
train_dataset = CustomDataset(image_root=train_path_images,
                              target_root=train_path_target,
                              mask_root=train_path_mask,
                              transform=train_transform)
test_dataset = CustomDataset(image_root=test_path,
                             transform = test_transform)


# check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)


# Put data into dataloaders
print('Loading DataLoaders')

# batch sizes
train_batch_size = 1
val_batch_size = 1
test_batch_size = 1

# split into train and val
train_size = int(len(train_dataset) * .7)
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

# load into dataloaders
train_loader = DataLoader(train_data, batch_size = train_batch_size, shuffle = False)
val_loader = DataLoader(val_data, batch_size = val_batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = test_batch_size, shuffle = False )

Retrieving Data
Device: cpu
Loading DataLoaders


Test Images

In [3]:
test_model2 = UNet(init_channels = 3,
            filters = 64,
            output_channels = 2,
            pool_mode = 'max',
            up_mode = 'upconv', 
            connection = 'cat',
            same_padding = True,
            use_batchnorm = True,
            conv_layers_per_block = 2,

            )

test_model2(x)

In [51]:
test_model = UNet2(init_channels = 3,
            filters = 64,
            output_channels = 2,
            model_depth = 4,
            pool_mode = 'max',
            up_mode = 'upconv', 
            connection = 'cat',
            same_padding = True,
            use_batchnorm = True,
            use_dropblock = True,
            block_size = 7,
            drop_prob = .1,
            conv_layers_per_block = 2,
            activation_fcn = 'leaky_relu'
            )


In [52]:
x = torch.rand(1, 3, 64, 64)
test_model(x)

Step 0 conv:  torch.Size([1, 64, 64, 64])
Step 0 pool:  torch.Size([1, 64, 32, 32])
Step 1 conv:  torch.Size([1, 128, 32, 32])
Step 1 pool:  torch.Size([1, 128, 16, 16])
Step 2 conv:  torch.Size([1, 256, 16, 16])
Step 2 pool:  torch.Size([1, 256, 8, 8])
Step 3 conv:  torch.Size([1, 512, 8, 8])
Step 3 pool:  torch.Size([1, 512, 4, 4])
Step conn:  torch.Size([1, 1024, 4, 4])
Step 0 upsample:  torch.Size([1, 512, 8, 8])
Step 0 conv:  torch.Size([1, 512, 8, 8])
Step 1 upsample:  torch.Size([1, 256, 16, 16])
Step 1 conv:  torch.Size([1, 256, 16, 16])
Step 2 upsample:  torch.Size([1, 128, 32, 32])
Step 2 conv:  torch.Size([1, 128, 32, 32])
Step 3 upsample:  torch.Size([1, 64, 64, 64])
Step 3 conv:  torch.Size([1, 64, 64, 64])
Step final:  torch.Size([1, 2, 64, 64])


tensor([[[[0.6866, 0.7877, 0.6736,  ..., 0.7460, 0.5567, 0.4887],
          [0.3583, 0.7436, 0.7284,  ..., 0.5950, 0.5157, 0.6511],
          [0.8029, 0.5866, 0.6310,  ..., 0.3194, 0.4971, 0.6713],
          ...,
          [0.5362, 0.3626, 0.3731,  ..., 0.4477, 0.3831, 0.3681],
          [0.5097, 0.2530, 0.5429,  ..., 0.3628, 0.4327, 0.4570],
          [0.4630, 0.5374, 0.8027,  ..., 0.2714, 0.3960, 0.4888]],

         [[0.5643, 0.1633, 0.1263,  ..., 0.2841, 0.3024, 0.2044],
          [0.3464, 0.3699, 0.5466,  ..., 0.4904, 0.5318, 0.3676],
          [0.6176, 0.2444, 0.6088,  ..., 0.6043, 0.3708, 0.3865],
          ...,
          [0.4840, 0.1896, 0.7436,  ..., 0.4849, 0.3944, 0.1769],
          [0.5861, 0.5546, 0.5197,  ..., 0.3790, 0.5772, 0.6016],
          [0.6396, 0.5695, 0.6446,  ..., 0.2827, 0.3751, 0.1732]]]],
       grad_fn=<SigmoidBackward>)

In [49]:
test_model

UNet2(
  (_dropblock): DropBlock2D()
  (_act_fcn): LeakyReLU(negative_slope=0.01)
  (down_blocks): ModuleList(
    (0): ModuleList(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (1): DropBlock2D()
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): LeakyReLU(negative_slope=0.01)
        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (5): DropBlock2D()
        (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (7): LeakyReLU(negative_slope=0.01)
      )
      (1): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ModuleList(
      (0): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (1): Dro

In [ ]:
for i in range(3, -1, -1):
    print(i)

In [ ]:
test_model.__dict__

In [ ]:
for i in enumerate(test_model.children()):
    print(i)

In [ ]:
import os
def load_model(model, path):
    model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    return model


In [ ]:
path = 'trains/trains2/models/epoch500/unet.pth'

In [ ]:
test_model = load_model(test_model, path)

loss_fn = nn.BCELoss()
device = 'cpu'

val_loss = val_epoch_debug(
                     network=test_model,
                     loss_fn=loss_fn,
                     dataloader=val_loader,
                     device=device,
                     use_mask=True)

In [ ]:
from utils.utils_metrics import *

In [ ]:
final_test_metrics(network=test_model,
                  val_dataloader=val_loader,
                  test_dataloader=test_loader,
                  train_losses=np.arange(0, 10, .1),
                  val_losses=np.arange(0, 20, .2),
                    confusion_threshold = .05,
                  device=device,
                  save=True,
                  save_path='./metrics/test3')

In [ ]:
z = torch.

In [ ]:
x = torch.rand(5, 5) + 1
x

In [ ]:
x.ge(1-.25).count_nonzero()


In [ ]:
x.ge(1 + .25) & x.lt(2-.25)

In [ ]:
nn.ModuleList([nn.ModuleList((nn.Sequential(nn.Conv2d(1,1,3)),nn.Sequential(nn.Conv2d(1, 1, 3)))),nn.ModuleList([nn.Sequential(nn.Conv2d(1,1,3)),nn.Sequential(nn.Conv2d(1, 1, 3))])])

In [ ]:
x = [None, nn.ReLU()]

In [ ]:
x

In [ ]:
nn.Sequential(*x)

In [ ]:
x.append(None)

In [ ]:
x